<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#版本关系" data-toc-modified-id="版本关系-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>版本关系</a></span></li><li><span><a href="#Unsupported-model-IR-version:-9,-max-supported-IR-version:-8" data-toc-modified-id="Unsupported-model-IR-version:-9,-max-supported-IR-version:-8-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Unsupported model IR version: 9, max supported IR version: 8</a></span></li><li><span><a href="#forward()-takes-2-positional-arguments-but-3-were-given" data-toc-modified-id="forward()-takes-2-positional-arguments-but-3-were-given-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>forward() takes 2 positional arguments but 3 were given</a></span></li><li><span><a href="#from-torch.onnx.symbolic_registry-import-register_op" data-toc-modified-id="from-torch.onnx.symbolic_registry-import-register_op-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>from torch.onnx.symbolic_registry import register_op</a></span></li><li><span><a href="#为torchscript算子添加映射" data-toc-modified-id="为torchscript算子添加映射-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>为torchscript算子添加映射</a></span><ul class="toc-item"><li><span><a href="#No-Op-registered-for-DeformConv-with-domain_version-of-14" data-toc-modified-id="No-Op-registered-for-DeformConv-with-domain_version-of-14-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>No Op registered for DeformConv with domain_version of 14</a></span></li><li><span><a href="#对应warning" data-toc-modified-id="对应warning-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>对应warning</a></span></li></ul></li><li><span><a href="#自定义torch.autograd.Function算子" data-toc-modified-id="自定义torch.autograd.Function算子-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>自定义torch.autograd.Function算子</a></span></li></ul></div>

## 版本关系

最重要的是torch, onnx和onnxruntime的版本要对得上，不然会出现奇奇怪怪的问题。目前可用的版本是：
+ torch: 1.8.1
+ onnx: 1.13.1
+ onnxruntime: 1.14.1

并且网上有人说需要先导入onnx再导入torch

+ https://discuss.pytorch.org/t/tutorial-for-torch-onnx-fails/182023

## Unsupported model IR version: 9, max supported IR version: 8

使用的onnx版本为1.14.0，这个版本导出的IR版本为9。onnxruntime的版本是1.14.1，这个版本最高支持IR8版本。将onnx降低为1.13.1后重新导出模型即解决该问题。

+ onnx各种版本对应关系参考：https://github.com/onnx/onnx/blob/main/docs/Versioning.md
+ onnx和onxruntime版本对应关系：https://github.com/microsoft/onnxruntime/blob/master/docs/Versioning.md

## forward() takes 2 positional arguments but 3 were given

参数是list，不是tuple:
```
# error
torch.onnx.export(cond_model, (a, b), "const_op.onnx")
# fix
torch.onnx.export(cond_model, [a, b], "const_op.onnx")
```

##  from torch.onnx.symbolic_registry import register_op

这个函数在1.13.1版本中被修改为`from torch.onnx import register_custom_op_symbolic`

## 为torchscript算子添加映射

### No Op registered for DeformConv with domain_version of 14

```python
def deformable_conv_symbolic(g, inx, weight, offset, mask, bias, stride_h, stride_w, pad_h, pad_w, dilation_h, dilation_w, groups, offset_groups, use_mask):
    return g.op("DeformConv", inx, weight, offset)
```

DeformConv算子从version 19才开始支持，而torch 1.13.1最高支持version 14，所以找不到对应的DeformConv算子。修改代码如下：
```python
def deformable_conv_symbolic(g, inx, weight, offset, mask, bias, stride_h, stride_w, pad_h, pad_w, dilation_h, dilation_w, groups, offset_groups, use_mask):
    return g.op("custom::deform_conv2d", inx, weight, offset)
```

### 对应warning
> /Users/yangqj/miniconda3/envs/py37/lib/python3.7/site-packages/torchvision/ops/deform_conv.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
>   if n_offset_grps == 0:
> /Users/yangqj/miniconda3/envs/py37/lib/python3.7/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: The shape inference of custom::deform_conv2d type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1888.)
>   _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
> /Users/yangqj/miniconda3/envs/py37/lib/python3.7/site-packages/torch/onnx/utils.py:688: UserWarning: The shape inference of custom::deform_conv2d type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1888.)
>   graph, params_dict, GLOBALS.export_onnx_opset_version
> /Users/yangqj/miniconda3/envs/py37/lib/python3.7/site-packages/torch/onnx/utils.py:1179: UserWarning: The shape inference of custom::deform_conv2d type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1888.)
>   graph, params_dict, GLOBALS.export_onnx_opset_version

torchscript算子的符号函数要求指明输入参数的类型，修改代码如下：
```python
@parse_args("v", "v", "v", "v", "v", "i", "i", "i", "i", "i", "i", "i", "i", "none") 
def deformable_conv_symbolic(g, inx, weight, offset, mask, bias, stride_h, stride_w, pad_h, pad_w, dilation_h, dilation_w, groups, offset_groups, use_mask):
    # 需要与ONNX中定义的算子输入保持一致
    return g.op("custom::deform_conv2d", inx, weight, offset)
```

## 自定义torch.autograd.Function算子